In [1]:
# HMDA Data Browser API
# https://cfpb.github.io/hmda-platform/#data-browser-api

# Import the libraries we need
import pandas as pd
import numpy as np

import re
import random

import requests
from requests import get
from bs4 import BeautifulSoup
from requests.compat import urljoin
#for creating a user based URL, 'urljoin' is used

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

import datetime
from datetime import timezone, time
from datetime import timedelta
import time

import matplotlib as plt
import matplotlib.pyplot as plt  # To visualize
from matplotlib.colors import LinearSegmentedColormap
# Import seaborn
import seaborn as sns

import json
from pandas.io.json import json_normalize

#api_url = "https://ffiec.cfpb.gov/v2/data-browser-api/view/aggregations?states=MD&years=2018&actions_taken=5,6"

#csv_url = "https://ffiec.cfpb.gov/v2/data-browser-api/view/csv?states=ND,SD,NE,KS,MN,IA,MO,WI,IL,IN,MI,OH&years=2019"

us_states = "AL,AK,AZ,AR,CA,CO,CT,DE,DC,FL,GA,HI,ID,IL,IN,IA,KS,KY,LA,ME,MD,MA,MI,MN,MS,MO,MT,NE,NV,NH,NJ,NM,NY,NC,ND,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY,AS,GU,MP,PR,VI,UM,FM,MH,PW"
#us_states = "AL"

year = "2020"


#us_states = "AL,AK"
#csv_url = "https://ffiec.cfpb.gov/v2/data-browser-api/view/csv?states=ND,SD,NE,KS,MN,IA,MO,WI,IL,IN,MI,OH&years=2019"
#us_states = "AK,AL"

csv_url = "https://ffiec.cfpb.gov/v2/data-browser-api/view/csv?states=%s&years=%s"%(us_states,year)

print(csv_url)


https://ffiec.cfpb.gov/v2/data-browser-api/view/csv?states=AL,AK,AZ,AR,CA,CO,CT,DE,DC,FL,GA,HI,ID,IL,IN,IA,KS,KY,LA,ME,MD,MA,MI,MN,MS,MO,MT,NE,NV,NH,NJ,NM,NY,NC,ND,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY,AS,GU,MP,PR,VI,UM,FM,MH,PW&years=2020


In [2]:
# To get JSON data from the API
# https://stackoverflow.com/questions/36240495/bound-method-response-json-of-response-200/36240505


#HMDA_Data = pd.read_csv(csv_url)

#Dropping row after Loan Amount (from row 23 to 99)
# https://www.geeksforgeeks.org/how-to-drop-one-or-multiple-columns-in-pandas-dataframe/
# How to drop columns in panda
#start = time.time()
#read data in chunks of 10000 rows at a time
#chunk = pd.read_csv(csv_url,chunksize=10000)

#chunk.drop(chunk.iloc[:, 22:], inplace = True, axis = 1)

#end = time.time()
#print("Read csv with chunks: ",(end-start),"sec")
#HMDA_Data = pd.concat(chunk)




#HMDA_Data.drop(HMDA_Data.iloc[:, 22:], inplace = True, axis = 1)

#HMDA_Data.head()





In [3]:
# Load the same CSV file 10X times faster and with 10X less memory
#https://towardsdatascience.com/%EF%B8%8F-load-the-same-csv-file-10x-times-faster-and-with-10x-less-memory-%EF%B8%8F-e93b485086c7

# Select only the required columns

#req_cols = ["activity_year","lei","state_code","county_code","conforming_loan_limit","derived_loan_product_type","derived_dwelling_category","derived_ethnicity","derived_race","derived_sex","action_taken","purchaser_type","preapproval","loan_type","loan_purpose","lien_status","reverse_mortgage","open-end_line_of_credit","business_or_commercial_purpose","loan_amount"]

req_cols = ["activity_year","lei","state_code","derived_loan_product_type","action_taken","loan_type","loan_purpose","lien_status","derived_race","loan_amount"]
#req_cols = ["activity_year","lei","state_code","loan_amount","action_taken"]

start = time.time()
#read data in chunks of 10000 rows at a time
chunk = pd.read_csv(csv_url,usecols=req_cols,chunksize=10000)

#chunk.drop(chunk.iloc[:, 22:], inplace = True, axis = 1)

end = time.time()
print("Read csv with chunks: ",(end-start),"sec")
HMDA_Data = pd.concat(chunk)


HMDA_Data = HMDA_Data.rename( columns= {'lei':'LEI'})

len(HMDA_Data)
#HMDA_Data.head()

# https://www.kite.com/python/answers/how-to-get-a-pivot-table-with-counts-of-unique-column-values-from-a-pandas-dataframe-in-python



Read csv with chunks:  2592.1543254852295 sec


25361306

In [5]:
HMDA_Data['action_taken'] = HMDA_Data['action_taken'].astype(str)
HMDA_Data['loan_type'] = HMDA_Data['loan_type'].astype(str)
HMDA_Data['loan_purpose'] = HMDA_Data['loan_purpose'].astype(str)
HMDA_Data['lien_status'] = HMDA_Data['lien_status'].astype(str)


HMDA_Data['action_taken'] = HMDA_Data['action_taken'].replace({'1':'Loan Originated',
                                                               '2':'Application approved but not accepted',
                                                               '3': 'Application denied',
                                                               '4':'Application withdrawn by applicant',
                                                               '5':'File closed for incompleteness',
                                                               '6':'Purchased loan',
                                                               '7':'Preapproval request denied',
                                                               '8':'Preapproval request approved but not accepted'})


HMDA_Data['loan_type'] = HMDA_Data['loan_type'].replace({'1' : 'Conventional',
                                                         '2' : 'FHA',
                                                         '3' : 'VA',
                                                         '4' : 'USDA'})


HMDA_Data['loan_purpose'] = HMDA_Data['loan_purpose'].replace({'1' : 'Home purchase',
                                                               '2' : 'Home improvement',
                                                               '31' : 'Refinancing',
                                                               '32' : 'Cash-out refinancing',
                                                               '4' : 'Other purpose',
                                                               '5' : 'Not applicable'})


HMDA_Data['lien_status'] = HMDA_Data['lien_status'].replace({'1' : 'Secured by a first lien',
                                                             '2' : 'Secured by a subordinate lien'})


#HMDA_Data['construction_method'] = HMDA_Data['construction_method'].replace({'1' : 'Site-built',
#                                                                             '2' : 'Manufactured home'})

#HMDA_Data.head()
print(HMDA_Data.head(5))

   activity_year                   LEI state_code derived_loan_product_type  \
0           2020  254900AX110CHF6FVV28         NV   Conventional:First Lien   
1           2020  254900AX110CHF6FVV28         TX            FHA:First Lien   
2           2020  254900AX110CHF6FVV28         TX   Conventional:First Lien   
3           2020  254900AX110CHF6FVV28         TX            FHA:First Lien   
4           2020  254900AX110CHF6FVV28         TX            FHA:First Lien   

                derived_race                        action_taken  \
0                      White  Application withdrawn by applicant   
1                      White  Application withdrawn by applicant   
2                      Asian                     Loan Originated   
3                      White                     Loan Originated   
4  Black or African American  Application withdrawn by applicant   

      loan_type   loan_purpose              lien_status  loan_amount  
0  Conventional  Home purchase  Secured by a 

In [9]:
#HMDA_Data_pivot = pd.pivot_table(data = HMDA_Data, index = ['LEI'], columns=['activity_year'], aggfunc=len, fill_value=0)


HMDA_Data_pivot_action_taken = pd.pivot_table(data = HMDA_Data, index = ['LEI'], columns = ['action_taken'],aggfunc={'state_code' : 'count', 'loan_amount' : 'sum'}, fill_value=0)
HMDA_Data_pivot_action_taken = HMDA_Data_pivot_action_taken.rename( columns= {'state_code':'Total Number of Mortgages', 'loan_amount':'Total Loan Amount'})



HMDA_Data_pivot_loan_type = pd.pivot_table(data = HMDA_Data, index = ['LEI'], columns = ['loan_type'],aggfunc={'state_code' : 'count', 'loan_amount' : 'sum'}, fill_value=0)
HMDA_Data_pivot_loan_type = HMDA_Data_pivot_loan_type.rename( columns= {'state_code':'Total Number of Mortgages', 'loan_amount':'Total Loan Amount'})



HMDA_Data_pivot_loan_purpose = pd.pivot_table(data = HMDA_Data, index = ['LEI'], columns = ['loan_purpose'],aggfunc={'state_code' : 'count', 'loan_amount' : 'sum'}, fill_value=0)
HMDA_Data_pivot_loan_purpose = HMDA_Data_pivot_loan_purpose.rename( columns= {'state_code':'Total Number of Mortgages', 'loan_amount':'Total Loan Amount'})



HMDA_Data_pivot_derived_race = pd.pivot_table(data = HMDA_Data, index = ['LEI'], columns = ['derived_race'],aggfunc={'state_code' : 'count', 'loan_amount' : 'sum'}, fill_value=0)
HMDA_Data_pivot_derived_race = HMDA_Data_pivot_derived_race.rename( columns= {'state_code':'Total Number of Mortgages', 'loan_amount':'Total Loan Amount'})



HMDA_Data_pivot_derived_loan_product_type = pd.pivot_table(data = HMDA_Data, index = ['LEI'], columns = ['derived_loan_product_type'],aggfunc={'state_code' : 'count', 'loan_amount' : 'sum'}, fill_value=0)
HMDA_Data_pivot_derived_loan_product_type = HMDA_Data_pivot_derived_loan_product_type.rename( columns= {'state_code':'Total Number of Mortgages', 'loan_amount':'Total Loan Amount'})





#Now we will convert the pivot table back to dataframe. For that we have to drop the empty column and then reset the index
#Use 'rename_axis(None, axis=1)' with reset_index() to remove any categories you might have created while using pivot table

#HMDA_Data_pivot.columns = HMDA_Data_pivot.columns.droplevel(0)
#HMDA_Data_pivot = HMDA_Data_pivot.reset_index().rename_axis(None, axis=1) 





#HMDA_Data_pivot.drop(HMDA_Data_pivot.iloc[:, 3:], inplace = True, axis = 1)


#HMDA_Data_pivot.head()

In [10]:
#Now we use vllookup to get LEI against HMDA count

#https://www.geeksforgeeks.org/how-to-do-a-vlookup-in-python-using-pandas/

lei_data_us = pd.read_csv("D:\\HMDA Data Analysis\\LEI Database USA.csv")

lei_data_us.head()

HMDA_Count_action_taken = pd.merge(lei_data_us, HMDA_Data_pivot_action_taken, on ='LEI', how ='inner')
HMDA_Count_action_taken.to_csv("D:\\HMDA Data Analysis\\HMDA Database %s"%year+"_All_By_Action_Taken.csv")

HMDA_Count_loan_type = pd.merge(lei_data_us, HMDA_Data_pivot_loan_type, on ='LEI', how ='inner')
HMDA_Count_loan_type.to_csv("D:\\HMDA Data Analysis\\HMDA Database %s"%year+"_All_By_loan_type.csv")

HMDA_Count_loan_purpose = pd.merge(lei_data_us, HMDA_Data_pivot_loan_purpose, on ='LEI', how ='inner')
HMDA_Count_loan_purpose.to_csv("D:\\HMDA Data Analysis\\HMDA Database %s"%year+"_All_By_loan_purpose.csv")

HMDA_Count_derived_race = pd.merge(lei_data_us, HMDA_Data_pivot_derived_race, on ='LEI', how ='inner')
HMDA_Count_derived_race.to_csv("D:\\HMDA Data Analysis\\HMDA Database %s"%year+"_All_By_derived_race.csv")

HMDA_Count_derived_loan_product_type = pd.merge(lei_data_us, HMDA_Data_pivot_derived_loan_product_type, on ='LEI', how ='inner')
HMDA_Count_derived_loan_product_type.to_csv("D:\\HMDA Data Analysis\\HMDA Database %s"%year+"_All_By__derived_loan_product_type.csv")



#HMDA_Count.head()
#len(HMDA_Count)

C:\Users\91903\AppData\Roaming\Python\Python38\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
